# Лабораторная работа #2

Подготовка: подключаем google drive

In [53]:
from google.colab import drive
drive.mount ('/content/drive')

file_path = '/content/drive/MyDrive/Colab Notebooks/ITSem4Lab2/Laptop_price.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import pandas as pd
import hashlib
import re
from cryptography.fernet import Fernet

In [55]:
# 1. Загрузка данных
try:
    df = pd.read_csv(file_path)  # Читаем CSV-файл в DataFrame (таблицу данных)
except Exception as e:
    print(f"Ошибка загрузки файла: {e}")

Функции

In [56]:
# 2. Проверка на CSV Injection (внедрение вредоносного кода через CSV)
#   - Проверяем, не начинаются ли текстовые значения в столбцах с опасных символов (=, +, -, @).
#   - Такие символы могут использоваться для выполнения кода в Excel и других программах, открывающих CSV.
def check_csv_injection(df):
    dangerous_chars = ('=', '+', '-', '@')  # Символы, которые могут использоваться для атаки CSV Injection

    # Проходимся по всем столбцам, содержащим строковые данные ('object')
    for col in df.select_dtypes(include=['object']).columns:
        # Проверяем, есть ли в столбце значения, начинающиеся с опасных символов
        if df[col].astype(str).apply(lambda x: x.lstrip().startswith(dangerous_chars)).any():     # К аргументу в теле лямбда функции добавили метод удаления пробелов перед методом .startwith()
            print(f"Обнаружена потенциальная CSV-инъекция в столбце {col}!")
        else:
            print(f"Столбец {col} безопасен.")  # Если столбец не содержит уязвимостей

# 3. Фильтрация данных от SQL-инъекций и XSS-атак
#   - SQL-инъекции: попытки внедрения SQL-кода в запросы к базе данных.
#   - XSS (Cross-Site Scripting): внедрение вредоносного JavaScript-кода на веб-страницы.
#   - Эта функция пытается предотвратить эти атаки, заменяя опасные конструкции на "[BLOCKED]".
def clean_input(value):
    # Преобразуем значение в нижний регистр для регистронезависимого поиска
    value = str(value).lower()

    # Список SQL-ключевых слов (команд), которые могут быть использованы для инъекций
    sql_keywords = ["select", "drop", "delete", "insert", "update", "alter", "union"]

    # Регулярные выражения для поиска XSS-атак (JavaScript)
    xss_patterns = [r'<script.*?>.*?</script>', r'javascript:.*', r'onerror=.*']

    # Проверяем наличие SQL-ключевых слов в значении
    for keyword in sql_keywords:
        if keyword in value:
            return "[BLOCKED]"  # Блокируем, если найдено SQL-ключевое слово

    # Проверяем наличие XSS-паттернов в значении
    for pattern in xss_patterns:
        if re.search(pattern, value):
            return "[BLOCKED]"  # Блокируем, если найден XSS-паттерн

                                                                                                  # Проверяем наличие паттерна комментария ("--")
    if "--" in value:
        return "[BLOCKED]"  # Блокируем, если найден паттерн комментария

    return value  # Если ничего опасного не найдено, возвращаем исходное значение

# 4. Хеширование столбца с ценами (SHA-256)
#   - Хеширование используется для защиты конфиденциальных данных (например, паролей).
#   - В данном случае мы хешируем цены, чтобы скрыть их реальные значения.
def hash_price(price):
    return hashlib.sha256(str(price).encode()).hexdigest()  # Вычисляем SHA-256 хеш

# 5. Шифрование данных (например, цены ноутбуков)
#   - Шифрование также скрывает данные, но их можно расшифровать, имея ключ.
#   - Используем библиотеку cryptography.fernet для шифрования.
# Генерируем ключ для шифрования (важно хранить его в секрете)
key = Fernet.generate_key()
cipher = Fernet(key)

# Функция для шифрования цены
def encrypt_price(price):
    return cipher.encrypt(str(price).encode()).decode()  # Шифруем цену

# Функция для расшифровки цены
def decrypt_price(encrypted_price):
    return cipher.decrypt(encrypted_price.encode()).decode()  # Расшифровываем цену

Проверка пробелов csv-инъекций

In [57]:
data_test1 = {'col1': ['Тут что-то важное', ' какие-то данные', ' +3-4'],
              'col2': ['кверти123', '  =1+2', '   больше пробелов'],
              'col3': [1, 2, 3],
              'col4': ['  @а тут таб', 'а тут нет таба', 73],
              'col5': ['  тут безопасный таб', 'mymail@elektromail.internet', ' Акция 1+1=11']}
df_test1 = pd.DataFrame(data_test1)

check_csv_injection(df_test1)

Обнаружена потенциальная CSV-инъекция в столбце col1!
Обнаружена потенциальная CSV-инъекция в столбце col2!
Обнаружена потенциальная CSV-инъекция в столбце col4!
Столбец col5 безопасен.


Проверка SQL паттерна комментария

In [58]:
data_test2 = {'col1': ['алерт скрипт', '<script>alert("XSS")</script>', 'string'],
              'col2': ['алерт яваскрипт', 'javascript:alert(1)', 'ok'],
              'col3': [1, 2, 3],
              'col4': ['селект фром юзерс', 'select * from users', 73],
              'col5': ['Коммент в середине строки', 'Это не коммент -- честночестно', '45'],
              'col6': ['дропаем юзеров', 'drop table users', 3],
              'col7': ['коммент', '-- comment', '?'],
              'col8': ['еще и с пробелом', ' -- comment with very long buton on keybord', 'текст']}
df_test2 = pd.DataFrame(data_test2)

# Применяем функцию clean_input ко всем элементам DataFrame
df_test2 = df_test2.applymap(lambda x: clean_input(x) if isinstance(x, str) else x)
print("Фильтрация данных завершена.")
print(df_test2)

Фильтрация данных завершена.
           col1             col2  col3               col4  \
0  алерт скрипт  алерт яваскрипт     1  селект фром юзерс   
1     [BLOCKED]        [BLOCKED]     2          [BLOCKED]   
2        string               ok     3                 73   

                        col5            col6       col7              col8  
0  коммент в середине строки  дропаем юзеров    коммент  еще и с пробелом  
1                  [BLOCKED]       [BLOCKED]  [BLOCKED]         [BLOCKED]  
2                         45               3          ?             текст  


<ipython-input-58-e73892393a26>:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_test2 = df_test2.applymap(lambda x: clean_input(x) if isinstance(x, str) else x)


Вызов функций

In [59]:
# Вызываем функцию проверки
check_csv_injection(df)

Столбец Brand безопасен.


In [60]:
# Применяем функцию clean_input ко всем элементам DataFrame
df = df.applymap(lambda x: clean_input(x) if isinstance(x, str) else x)
print("Фильтрация данных завершена.")

Фильтрация данных завершена.


<ipython-input-60-494beb69d310>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: clean_input(x) if isinstance(x, str) else x)


In [61]:
df.head(5)

,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price
0,asus,3.830296,16,512,11.185147,2.641094,17395.093065
1,acer,2.912833,4,1000,11.311372,3.260012,31607.605919
2,lenovo,3.241627,4,256,11.853023,2.029061,9291.023542
3,acer,3.806248,16,512,12.280360,4.573865,17436.728334
4,acer,3.268097,32,1000,14.990877,4.193472,32917.990718


In [62]:
# Применяем функцию хеширования к столбцу 'Price'
df['Price_Hashed'] = df['Price'].apply(hash_price)
print("Столбец с хешированными ценами добавлен.")

Столбец с хешированными ценами добавлен.


In [63]:
df.head(5)

,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price,Price_Hashed
0,asus,3.830296,16,512,11.185147,2.641094,17395.093065,e33562234b73b4ecad16901885fd503f2c4c20996e22d9...
1,acer,2.912833,4,1000,11.311372,3.260012,31607.605919,80fe437d4a9c57c4038198b7cf32f93b9f957bb705b8e1...
2,lenovo,3.241627,4,256,11.853023,2.029061,9291.023542,6254a5a18d60d796eb464283ef38f9417e12885a07742e...
3,acer,3.806248,16,512,12.280360,4.573865,17436.728334,1f2910923c33a53b30bc7e6eb731307eb2e2366b3811b6...
4,acer,3.268097,32,1000,14.990877,4.193472,32917.990718,f90738a55c90c1e31a4fb6d35f92306ff03a87001a9408...


In [64]:
# Применяем функцию шифрования к столбцу 'Price'
df['Price_Encrypted'] = df['Price'].apply(encrypt_price)
print("Столбец с зашифрованными ценами добавлен.")

df['Price_Encrypted'].head(5) # Выводим 5 зашифрованных цен

Столбец с зашифрованными ценами добавлен.


,Price_Encrypted
0,gAAAAABoIkbQmmm29r0mJDN0Ons5cmqlWRcDObSgt2n5bp...
1,gAAAAABoIkbQUOCPX0mbZYqRNIwTZsf4aZ7p33g9-kAuK1...
2,gAAAAABoIkbQTpU2l4takPNrqRgIFDQWfkMg-FuM6R4tdX...
3,gAAAAABoIkbQIgMRZ57-Ws972vTbWvjsi8kX9Oxttu4uRO...
4,gAAAAABoIkbQ0DltoCkJtn1W5BXSZ-HhNX_3pjFdpIPZLo...


Сохранение обработанных данных в новый CSV-файл

In [65]:
output_path = "/content/drive/MyDrive/Colab Notebooks/ITSem4Lab2/Laptop_price_secured.csv"  # Имя выходного файла
df.to_csv(output_path, index=False)  # Сохраняем DataFrame в CSV (без индексов)
print(f"Обработанный файл сохранен: {output_path}")  # Выводим сообщение об успехе

Обработанный файл сохранен: /content/drive/MyDrive/Colab Notebooks/ITSem4Lab2/Laptop_price_secured.csv


(Рас)шифровка столбка RAM_Size

In [66]:
# Применяем функцию шифрования к столбцу 'RAM'
df['RAM_Encrypted'] = df['RAM_Size'].apply(encrypt_price)
print("Столбец с зашифрованными памятями добавлен.")

df['RAM_Decrypted'] = df['RAM_Encrypted'].apply(decrypt_price)


df.head(5) # Выводим 5 строк

Столбец с зашифрованными памятями добавлен.


,Brand,Processor_Speed,RAM_Size,Storage_Capacity,Screen_Size,Weight,Price,Price_Hashed,Price_Encrypted,RAM_Encrypted,RAM_Decrypted
0,asus,3.830296,16,512,11.185147,2.641094,17395.093065,e33562234b73b4ecad16901885fd503f2c4c20996e22d9...,gAAAAABoIkbQmmm29r0mJDN0Ons5cmqlWRcDObSgt2n5bp...,gAAAAABoIkbXftWAGN4leRLS9wDtLLmNAoWQDNb5fXgsOs...,16
1,acer,2.912833,4,1000,11.311372,3.260012,31607.605919,80fe437d4a9c57c4038198b7cf32f93b9f957bb705b8e1...,gAAAAABoIkbQUOCPX0mbZYqRNIwTZsf4aZ7p33g9-kAuK1...,gAAAAABoIkbXDj1B2oYycLxy0xefkrw-hu-B8pJyFsZB46...,4
2,lenovo,3.241627,4,256,11.853023,2.029061,9291.023542,6254a5a18d60d796eb464283ef38f9417e12885a07742e...,gAAAAABoIkbQTpU2l4takPNrqRgIFDQWfkMg-FuM6R4tdX...,gAAAAABoIkbXrVGz-FLKjoPwsaptkBG7X1bWWRK9Eif305...,4
3,acer,3.806248,16,512,12.280360,4.573865,17436.728334,1f2910923c33a53b30bc7e6eb731307eb2e2366b3811b6...,gAAAAABoIkbQIgMRZ57-Ws972vTbWvjsi8kX9Oxttu4uRO...,gAAAAABoIkbX0g1QIyyvjV_cUm27_D4POhlDR20sMZOmIc...,16
4,acer,3.268097,32,1000,14.990877,4.193472,32917.990718,f90738a55c90c1e31a4fb6d35f92306ff03a87001a9408...,gAAAAABoIkbQ0DltoCkJtn1W5BXSZ-HhNX_3pjFdpIPZLo...,gAAAAABoIkbX5xTA6pYZ1zN_O9jkEDdvu__VTmgFRMdFgZ...,32


Сохраняем защифрованную RAM

In [67]:
output_path = "/content/drive/MyDrive/Colab Notebooks/ITSem4Lab2/Laptop_price_RAM.csv"  # Имя выходного файла
df.to_csv(output_path, index=False)  # Сохраняем DataFrame в CSV (без индексов)
print(f"Обработанный файл сохранен: {output_path}")  # Выводим сообщение об успехе

Обработанный файл сохранен: /content/drive/MyDrive/Colab Notebooks/ITSem4Lab2/Laptop_price_RAM.csv


Коммитим в гит



In [ ]:
from google.colab import userdata

!git --version

%cd /content/drive/MyDrive/Colab\ Notebooks/ITSem4Lab2

GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_REPOSITORY = 'ITSem4Lab2'
GITHUB_USERNAME = 'morganuk'

!git init
!git add Sem4Lab2.ipynb
!git add Laptop_price.csv
!git add Laptop_price_secured.csv
!git add Laptop_price_RAM.csv

!git config --global user.email "145021797+morganuk@users.noreply.github.com"
!git config --global user.name "Morgan"

REPOSITORY_URL = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USERNAME}/{GITHUB_REPOSITORY}.git"
!git remote add origin {REPOSITORY_URL}

In [73]:
!git commit -m "Добавлен Шифрование/расшифрование стобца RAM"

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Sem4Lab2.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [74]:
!git branch -M main       # Выбираем ветку
!git push -u origin main  # Пушим в удаленный репозиторий

Branch 'main' set up to track remote branch 'main' from 'origin'.
Everything up-to-date
